## News Mood

### Observations
* CBS has the most number positive tweets and BBC World has the most number of negative tweets
* CNN and New York Times tend towards tweeting more neutral tweets
* The tweets of CBS are becoming increasingly positive as the number of tweets increase

In [1]:
# Import Dependencies
import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [2]:
# Target Search Term
target_terms = ("@bbcworld","@cbs","@cnn","@foxnews","@nytimes")

# List to hold results
results_list = []
compound_list = []
positive_list = []
negative_list = []
neutral_list = []
source_account=[]
text=[]
date=[]
tweets_ago=[]

for target in target_terms:
    tweet_count=1
    # Loop through 5 pages
    for x in range(1,6):

        # Run search around each tweet
        public_tweets = api.user_timeline(target,page=x)

        # Loop through all tweets
        for tweet in public_tweets:

                # Run Vader Analysis on each tweet
                results = analyzer.polarity_scores(tweet["text"])

                # Add each value to the appropriate list
                source_account.append(tweet['user']['name'])
                text.append(tweet['text'])
                date.append(tweet['created_at'])
                compound_list.append(results["compound"])
                positive_list.append(results["pos"])
                negative_list.append(results["neu"])
                neutral_list.append(results["neg"])
                tweets_ago.append(tweet_count)
                tweet_count+=1

#Create dictionary from the lists
news={"Source Account":source_account,"Text":text,"Date":date,"Compound Score":compound_list,"Positive Score":positive_list,"Negative Score":negative_list,"Neutral Score":neutral_list,"Tweets Ago":tweets_ago}

#Create dataframe
news_df=pd.DataFrame(news).round(3)
news_df

Source Account                                               Text  \
0      BBC News (World)  The bean at the heart of America's trade war h...   
1      BBC News (World)  Toddler mauled by dog dies in Australia https:...   
2      BBC News (World)  The European populists monitoring 'sham' elect...   
3      BBC News (World)  Russia shocked by mysterious murder of three j...   
4      BBC News (World)  Officer wins suit over claim he was mocked for...   
5      BBC News (World)  The moment a passenger plane crashed in Mexico...   
6      BBC News (World)  New Zealand PM Jacinda Ardern returns from mat...   
7      BBC News (World)  RT @BBCNews: Passengers were forced to evacuat...   
8      BBC News (World)  Jewellery thefts that shocked the world https:...   
9      BBC News (World)  US hits Turkish ministers with sanctions over ...   
10     BBC News (World)  McDonald's serves pregnant Canadian cleaning f...   
11     BBC News (World)  Ethiopia's exiled patriarch Bishop Merkorios r...   
12     BBC News (World)  Shifted sands reveal Roman settlement in Alder...   
13     BBC News (World)  "We had just taken off then the plane fell" — ...   
14     BBC News (World)  French rappers Booba and Kaaris brawl at Paris...   
15     BBC News (World)  Chinese takeover of German firm Leifeld collap...   
16     BBC News (World)  In pictures: Zimbabwe election protesters clas...   
17     BBC News (World)  'Disappearance' of top Chinese actress Fan Bin...   
18     BBC News (World)  Trump demands Russia probe end 'right now' htt...   
19     BBC News (World)  Canada's Ontario government cuts basic income ...   
20     BBC News (World)  👣👣👣\n\n"Imagine, if you would, a world without...   
21     BBC News (World)  Tajikistan attack: Four cyclists mown down are...   
22     BBC News (World)  Zimbabwe election: MDC Alliance supporters in ...   
23     BBC News (World)  How likely are you to survive a plane crash? h...   
24     BBC News (World)  Jellyfish sting dozens as Germany and Sweden b...   
25     BBC News (World)  Greek wildfires: Failures revealed after Mati ...   
26     BBC News (World)  Stranded Russian climber 'hallucinated' during...   
27     BBC News (World)  Dengue fever outbreak stopped by special mosqu...   
28     BBC News (World)  Ex-warlord Jean-Pierre Bemba welcomed home to ...   
29     BBC News (World)  Trump claims Americans need ID to go shopping ...   
..                  ...                                                ...   
470  The New York Times  Facebook said that it had identified a politic...   
471  The New York Times  Police say a Connecticut man used dating apps ...   
472  The New York Times  RT @nytimesworld: 3 Russian journalists who we...   
473  The New York Times  "The touchpoint with ISIS is not by proximity ...   
474  The New York Times  RT @nytimesworld: “I don’t think the governmen...   
475  The New York Times  "Among everyone caught up in the #MeToo moveme...   
476  The New York Times  RT @nytopinion: Exploiting my family by having...   
477  The New York Times  "You started understanding the life of the peo...   
478  The New York Times  China’s Plan to Win Friends and Influence Incl...   
479  The New York Times  RT @PekingMike: Important story on some pushba...   
480  The New York Times  One Cambodian voter defaced his or her ballot ...   
481  The New York Times  There it was: Glowering red on the dashboard o...   
482  The New York Times  New Jersey, in a first-of-its-kind initiative,...   
483  The New York Times  RT @nytimesworld: Rumors are swirling in Tehra...   
484  The New York Times  News of Laos Dam Failure Didn’t Reach Them, bu...   
485  The New York Times  Florida has an algae problem, and it’s big. A ...   
486  The New York Times  His mother ran toward him, her heart racing. "...   
487  The New York Times  RT @nytopinion: Ida B. Wells, anti-lynching ch...   
488  The New York Times  An astrophysicist finds even more evidence for...   
489  The New York Times  RT @NY

In [3]:
#to csv file
news_df.to_csv("news.csv")

In [4]:
#Sentiment analysis of media tweets 
#Segregate sentiments according to the news channel
bbc_df=news_df.loc[news_df["Source Account"]=="BBC News (World)"]
cbs_df=news_df.loc[news_df["Source Account"]=="CBS"]
cnn_df=news_df.loc[news_df["Source Account"]=="CNN"]
fox_df=news_df.loc[news_df["Source Account"]=="Fox News"]
nyt_df=news_df.loc[news_df["Source Account"]=="The New York Times"]

#get today's date
today_date = datetime.datetime.now().strftime('%D')

#plot scatter plot
plt.figure(figsize=(8,6))
plot=plt.scatter(bbc_df["Tweets Ago"], bbc_df["Compound Score"],s=75, marker="o", facecolors="skyblue", edgecolors="black",label="BBC",alpha=0.75)
plot=plt.scatter(cbs_df["Tweets Ago"], cbs_df["Compound Score"],s=75, marker="o", facecolors="green", edgecolors="black",label="CBS",alpha=0.75)
plot=plt.scatter(cnn_df["Tweets Ago"], cnn_df["Compound Score"],s=75, marker="o", facecolors="red", edgecolors="black",label="CNN",alpha=0.75)
plot=plt.scatter(fox_df["Tweets Ago"], fox_df["Compound Score"],s=75, marker="o", facecolors="blue", edgecolors="black",label="Fox",alpha=0.75)
plot=plt.scatter(nyt_df["Tweets Ago"], nyt_df["Compound Score"],s=75, marker="o", facecolors="yellow", edgecolors="black",label="New York Times",alpha=0.75)

# Incorporate the other graph properties
plt.xlim(105,-5)
plt.legend(title="Media Sources",bbox_to_anchor = (1,1))
plt.title(f"Sentiment Analysis of Media Tweets ({today_date})")
plt.xlabel("Tweets Ago")
plt.ylabel("Tweet Polarity")
plt.grid()
plt.savefig("Sentiment Analysis of Media Tweets.png")


In [6]:
#Overall Media Sentiment Based on Twitter
#Average overall sentiment
average_sentiment=news_df.groupby('Source Account')['Compound Score'].mean()

#Plot bar graph 
x_axis = np.arange(len(average_sentiment))
xlabels = average_sentiment.index
plt.figure(figsize=(8,6))
plt.bar(x_axis, average_sentiment, tick_label = xlabels, color = ['lightskyblue', 'green', 'red', 'blue', 'yellow'], edgecolor="black",width=1)

#plot value for each bar
count = 0
for sentiment in average_sentiment:
    plt.text(count, sentiment+.001, str(round(sentiment,2)))
    count += 1

# Incorporate the other graph properties
plt.title(f"Overall Media Sentiment Based on Twitter ({today_date})")
plt.ylabel("Tweet Polarity")
plt.savefig("Overall Media Sentiment Based on Twitter.png")



